In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets
import plotly.express as px
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.nonparametric.smoothers_lowess import lowess
import numpy as np


In [40]:
df_fortaleza = pd.read_excel("https://www.sspds.ce.gov.br/wp-content/uploads/sites/24/2025/03/CVLI_2009-2024.xlsx", engine="openpyxl")

with open("source_data/populacao_2022.json", "r", encoding="utf-8") as f:
    populacao_2022 = json.load(f)

with open("source_data/bairros_por_ais.json", "r", encoding="utf-8") as f:
    bairros_por_ais = json.load(f)


In [41]:
populacao_total_ais = {
    ais: sum(populacao_2022.get(bairro, 0) for bairro in bairros)
    for ais, bairros in bairros_por_ais.items()
}

In [42]:
def agrupar_por_ais(df:pd.DataFrame, nome_ais:str):
    df_filtrada = df[df["AIS"] == nome_ais]
    df_agrupada = df_filtrada.groupby("Ano").size().reset_index(name="Total")
    return df_agrupada

In [43]:
def plot_serie_temporal(df_para_plot:pd.DataFrame, ais:str):
    df_para_plot = agrupar_por_ais(df_para_plot, ais)

    plt.figure(figsize=(16, 6))
    sns.barplot(data=df_para_plot,
                x="Ano",
                hue="Ano",
                y="Total",
                legend=False,
                palette="coolwarm"
                )
    plt.title(f"Quantidade de Casos por Ano da {ais}")
    plt.xlabel("Ano")
    plt.ylabel("Quantidade de Casos")
    plt.show()

In [44]:
def serie_temporal_interativa(df: pd.DataFrame, ais: str):
    # Agrupar os dados da AIS
    df = agrupar_por_ais(df, ais)
    numero_ais = ais.split()[1]
    # Obter a população da AIS
    populacao = (
        populacao_total_ais.get(str(int(numero_ais)))
        if numero_ais is not None
        else None
    )

    # Formatar o título com a população
    if populacao is not None:
        populacao_formatada = f"{populacao:,}".replace(",", ".")
        titulo = (
            f"Quantidade de Casos por Ano - {ais} "
            f"(População: {populacao_formatada} habitantes, de acordo com o Censo do IBGE 2022)"
        )
    else:
        titulo = f"Quantidade de Casos por Ano - {ais} (População)"

    
    fig = px.bar(
        df,
        x="Ano",
        y="Total",
        color="Total",
        title=titulo,
        labels={"Total": "Quantidade de Casos"},
        height=600
    )

    fig.update_layout(showlegend=False)
    fig.show()


In [45]:
def plotar_series_temporais(df: pd.DataFrame):
    # Filtra valores válidos
    df = df[df["AIS"] != "AIS não identificadas"]
    
    ais_options = sorted(df['AIS'].unique())
    dropdown = widgets.Dropdown(options=ais_options, description='AIS:')
    
    interact(lambda ais: serie_temporal_interativa(df, ais), ais=dropdown)

In [46]:
df_fortaleza["Data"] = pd.to_datetime(df_fortaleza["Data"], format="%Y-%m-%d", errors="coerce")

In [47]:
df_fortaleza["Ano"] = df_fortaleza["Data"].dt.year

In [48]:
plotar_series_temporais(df_fortaleza)

interactive(children=(Dropdown(description='AIS:', options=('AIS 01', 'AIS 02', 'AIS 03', 'AIS 04', 'AIS 05', …

In [49]:
def time_series_with_trend_interactive(
    df: pd.DataFrame,
    time_col: str = 'Data',
    size: tuple = (1000, 500),
    title: str = None,
    xlabel: str = None,
    ylabel: str = "Ocorrências",
    time_unit: str = 'Y',
    color_palette: str = "Blues",
    rotation: int = 45,
    trend_color: str = "crimson"
) -> None:
    """
    Plota uma série temporal interativa com linha de tendência suavizada.
    Versão corrigida e testada para DataFrames com coluna 'Data'.

    Parâmetros:
    -----------
    df : pd.DataFrame
        DataFrame de entrada contendo a coluna temporal
    time_col : str, optional
        Nome da coluna de tempo. Padrão: 'Data'
    size : tuple, optional
        Tamanho da figura em pixels. Padrão: (1000, 500)
    title : str, optional
        Título do gráfico
    xlabel : str, optional
        Rótulo do eixo x
    ylabel : str, optional
        Rótulo do eixo y. Padrão: "Ocorrências"
    time_unit : str, optional
        Unidade de tempo ('Y', 'M' ou 'D'). Padrão: 'Y'
    color_palette : str, optional
        Nome da paleta de cores do Plotly. Padrão: "Blues"
    rotation : int, optional
        Rotação dos rótulos do eixo x. Padrão: 45
    trend_color : str, optional
        Cor da linha de tendência. Padrão: "crimson"
    """
    from statsmodels.nonparametric.smoothers_lowess import lowess
    
    # Verificação da coluna temporal
    if time_col not in df.columns:
        available_cols = [col for col in df.columns if col.lower() in ['data', 'date', 'tempo', 'time']]
        suggestions = f" Talvez você queira usar: {available_cols[0]}" if available_cols else ""
        raise KeyError(
            f"Coluna temporal '{time_col}' não encontrada.{suggestions}\n"
            f"Colunas disponíveis: {list(df.columns)}"
        )

    # Processamento dos dados
    try:
        df_processed = get_time_group(df, time_col=time_col, time_unit=time_unit)
    except Exception as e:
        raise ValueError(f"Erro ao processar dados temporais: {str(e)}")

    # Agrupar os dados
    grouped = df_processed.groupby("time_group").size().reset_index(name="Total")
    grouped = grouped.sort_values("time_group")

    # Obter cores da paleta do Plotly corretamente
    try:
        colors = getattr(px.colors.sequential, color_palette)
    except AttributeError:
        colors = px.colors.sequential.Blues  # Fallback para Blues

    # Criar figura com subplots
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=("Distribuição Temporal", "Tendência Suavizada"),
        horizontal_spacing=0.1
    )

    try:
        palette = getattr(px.colors.sequential, color_palette)
        # Selecionar cores igualmente espaçadas da paleta para cada barra
        n_bars = len(grouped)
        colors = [palette[i % len(palette)] for i in range(n_bars)]
    except AttributeError:
        colors = px.colors.sequential.Blues  # Fallback para Blues
        colors = [colors[i % len(colors)] for i in range(len(grouped))]

    # Adicionar gráfico de barras
    fig.add_trace(
        go.Bar(
            x=grouped["time_group"],
            y=grouped["Total"],
            marker=dict(
                color=colors,  # Usar a lista de cores personalizada
                line=dict(color='rgba(0,0,0,0.3)', width=1)  # Borda sutil
            ),
            name="Ocorrências",
            hovertemplate="<b>%{x}</b><br>Ocorrências: %{y}<extra></extra>"
        ),
        row=1, col=1
    )

    # Preparar dados para tendência
    x = grouped["time_group"]
    y = grouped["Total"]

    # Converter para numérico se for datetime
    if pd.api.types.is_datetime64_any_dtype(x):
        x_num = x.astype(np.int64) // 10**9
        x_labels = x.dt.strftime('%Y-%m-%d' if time_unit == 'D' else '%Y-%m')
    else:
        x_num = x.astype(float)
        x_labels = x.astype(str)

    # Adicionar linha de tendência (LOWESS)
    low = lowess(y, x_num, frac=1/3)
    fig.add_trace(
        go.Scatter(
            x=x,
            y=low[:, 1],
            mode='lines',
            line=dict(color=trend_color, width=3),
            name='Tendência',
            hovertemplate="<b>%{x}</b><br>Tendência: %{y:.1f}<extra></extra>"
        ),
        row=1, col=2
    )


    # Adicionar pontos de dados
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=dict(color=trend_color, size=8),
            name='Dados',
            hovertemplate="<b>%{x}</b><br>Ocorrências: %{y}<extra></extra>",
            showlegend=False
        ),
        row=1, col=2
    )

    # Configurações de layout
    fig.update_layout(
        title_text=f"{title}<br><sup>Análise Temporal</sup>" if title else None,
        width=size[0],
        height=size[1],
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(l=50, r=50, b=80, t=80 if title else 50)
    )

    # Configurações dos eixos
    date_format = '%Y-%m-%d' if time_unit == 'D' else '%Y-%m' if time_unit == 'M' else None

    fig.update_xaxes(
        title_text=xlabel or "Período",
        row=1, col=1,
        tickangle=rotation if time_unit in ['M', 'D'] else 0,
        tickformat=date_format
    )

    fig.update_xaxes(
        title_text=xlabel or "Período",
        row=1, col=2,
        tickangle=rotation if time_unit in ['M', 'D'] else 0,
        tickformat=date_format
    )

    fig.update_yaxes(title_text=ylabel, row=1, col=1)
    fig.update_yaxes(title_text=ylabel, row=1, col=2)

    fig.show()

In [50]:
def get_time_group(df, time_col='date', time_unit='Y'):
    """
    Adiciona uma coluna de agrupamento temporal ao DataFrame.

    Parâmetros:
    -----------
    df : pd.DataFrame
        DataFrame de entrada
    time_col : str, optional
        Nome da coluna de tempo. Padrão: 'date'
    time_unit : str, optional
        Unidade de tempo ('Y' para ano, 'M' para mês, 'D' para dia).
        Padrão: 'Y'

    Retorna:
    --------
    pd.DataFrame
        DataFrame com a coluna 'time_group' adicionada (sempre numérica ou datetime)
    """
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])

    if time_unit == 'Y':
        df['time_group'] = df[time_col].dt.year
        df['time_group_str'] = df['time_group'].astype(str)  # Versão string para rótulos
    elif time_unit == 'M':
        df['time_group'] = df[time_col].dt.to_period('M').dt.to_timestamp()
        df['time_group_str'] = df['time_group'].dt.strftime('%Y-%m')
    elif time_unit == 'D':
        df['time_group'] = df[time_col].dt.normalize()
        df['time_group_str'] = df['time_group'].dt.strftime('%Y-%m-%d')
    else:
        raise ValueError("Unidade de tempo não suportada. Use 'Y', 'M' ou 'D'.")

    return df

In [51]:
# Exemplo com seus dados
time_series_with_trend_interactive(
    df=df_fortaleza,
    time_col='Data',  # Nome da sua coluna de data
    time_unit='Y',    # Agrupar por ano ('Y', 'M' ou 'D')
    title="Análise Temporal de Crimes",
    xlabel="Ano",
    ylabel="Número de Ocorrências",
    color_palette="Viridis"  # Paletas disponíveis: https://plotly.com/python/builtin-colorscales/
)

In [52]:
# Exemplo com seus dados
time_series_with_trend_interactive(
    df=df_fortaleza,
    time_col='Data',  # Nome da sua coluna de data
    time_unit='M',    # Agrupar por ano ('Y', 'M' ou 'D')
    title="Análise Temporal de Crimes",
    xlabel="Ano",
    ylabel="Número de Ocorrências",
    color_palette="Viridis"  # Paletas disponíveis: https://plotly.com/python/builtin-colorscales/
)